In [42]:
#Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [31]:
#Import data
fifteen = pd.read_csv("../data/2015.csv")
sixteen = pd.read_csv("../data/2016.csv")
seventeen = pd.read_csv("../data/2017.csv")
eighteen = pd.read_csv("../data/2018.csv")
nineteen = pd.read_csv("../data/2019.csv")

In [32]:
# Rename columns
seventeen = seventeen.rename(columns = {
    "Happiness.Rank":"Happiness Rank", "Economy..GDP.per.Capita.":"Economy (GDP per Capita)", 
    "Health..Life.Expectancy.":"Health (Life Expectancy)","Trust..Government.Corruption.":"Trust (Government Corruption)"
})

eighteen = eighteen.rename(columns = {
    "Country or region":"Country", "Overall rank":"Happiness Rank", "GDP per capita":"Economy (GDP per Capita)",
    "Social support":"Family", "Healthy life expectancy":"Health (Life Expectancy)", "Freedom to make life choices":"Freedom",
    "Perceptions of corruption":"Trust (Government Corruption)"
})
nineteen = nineteen.rename(columns = {
    "Country or region":"Country", "Overall rank":"Happiness Rank", "GDP per capita":"Economy (GDP per Capita)",
    "Social support":"Family", "Healthy life expectancy":"Health (Life Expectancy)", "Freedom to make life choices":"Freedom",
    "Perceptions of corruption":"Trust (Government Corruption)"
})

In [33]:

dfs = [fifteen, sixteen, seventeen, eighteen, nineteen]
for df in dfs:
    print(df.columns)

Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual'],
      dtype='object')
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual'],
      dtype='object')
Index(['Country', 'Happiness Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Generosity',
       'Trust (Government Corruption)', 'Dystopia.Residual'],
      dtype='object')
Index(['Happiness Rank', 'Country', 'Score', 'Economy (GDP per Capita)',
       'Family', 'Health (Life Expectancy)', 'Freedom', 'Gene

In [34]:
type(dfs)

list

In [35]:
df = pd.concat(dfs) # build pandas dataframe from list(dfs)
df = df.sort_values(by = ['Country']) # sort by country

In [36]:
# drop columns that are not necessary/helpful in analysis
df = df.drop(columns=['Happiness Score','Standard Error','Dystopia Residual','Lower Confidence Interval','Upper Confidence Interval','Happiness.Score','Whisker.high','Whisker.low','Dystopia.Residual','Score',])

df = df.reset_index() # reset the indext

# Drop Northern Cyprus because it is not recognised as a country (apart from by Turkey)
df = df.drop(index=df.iloc[np.where(df['Country'] == 'Northern Cyprus')].index)

df = df.drop(index=df.iloc[np.where(df['Country'] == 'Northern Macedonia')].index)

df.at[625,['Country']] = 'Somaliland Region'

In [37]:
type(df)

pandas.core.frame.DataFrame

In [38]:
# Pre-process countries
countries = list(set(df['Country'].values))
countries = sorted(countries)

In [39]:
# Drop countries with NaN's
new_regions = []
for i in range(len(countries)):
        country_index = np.where(df['Country'] == countries[i]) # Get df of current country
        current_country_df = df.iloc[country_index]
        if all(pd.isnull(current_country_df['Region'].values)): # if all null for regions, drop country from df
            country_index = np.where(df['Country'] == countries[i])[0]
            for vals in country_index:
                try:
                    df = df.drop(index = vals)
                except KeyError:
                    pass
        else:
            region_name_pos = np.where([type(x) == str for x in current_country_df['Region'].values])[0][0]
            region = current_country_df['Region'].values[region_name_pos]
            current_country_df = current_country_df.replace(np.nan, region)
            new_regions.append(current_country_df['Region'].values)

In [41]:
new_regions = list(new_regions)
k = 0
region_vals = []
for arr in new_regions:
    for vals in arr:
        region_vals.append(vals)
        k+=1
        
len(region_vals)
df['Region'] = region_vals
